In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext line_profiler

In [2]:
import pandas as pd
import numpy as np
import pulp
import random
import matplotlib.pyplot as plt
import sys
import os
sys.path.append(os.path.abspath("C:\\Users\\Christophe GOUDET\\Documents\\Programmes\\RoomOptimisation"))
from LibRoomOptimisation import CreatePersoData, CreateOfficeData

In [10]:
persoData = pd.read_csv('dum.csv')

In [11]:
persoData=persoData[persoData['isCodir']==0]
persoData=persoData[persoData['isGRC']==0]

In [12]:
persoData.groupby('etage').count()


,Unnamed: 0,Nom,rawPerso1,rawPerso2,rawPerso3,rawEtage,rawWindow,rawClim,rawSeul,rawPassage,...,weightPerso3,perso3,inService,isCodir,secure,isGRC,isFace,isAgathe,mur,phone
etage,,,,,,,,,,,,,,,,,,,,,
0,5,5,3,3,3,5,5,5,5,5,...,5,3,5,5,5,5,5,5,5,5
1,8,8,7,7,5,8,8,8,8,8,...,8,5,8,8,8,8,8,8,8,8
2,14,14,13,13,12,14,14,14,14,14,...,14,12,14,14,14,14,14,14,14,14
